In [13]:
!nvidia-smi

Wed Jan 17 12:05:47 2018       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 384.81                 Driver Version: 384.81                    |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:04:00.0 Off |                    0 |
| N/A   51C    P0    85W / 149W |  11171MiB / 11439MiB |     59%      Default |
+-------------------------------+----------------------+----------------------+
|   1  Tesla K80           Off  | 00000000:05:00.0 Off |                    0 |
| N/A   36C    P0    74W / 149W |   8312MiB / 11439MiB |     35%      Default |
+-------------------------------+----------------------+----------------------+
|   2  T

In [2]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [3]:
import os.path as osp

import fcn
import numpy as np
import skimage.io
import torch
from torch.autograd import Variable
import torchfcn
import tqdm

In [4]:
model_file = torchfcn.models.fcn32s.FCN32s.download()
root = osp.expanduser('~/data/datasets')

[/zfsauton/home/mbarnes1/data/models/pytorch/fcn32s_from_caffe.pth] Checking md5 (8acf386d722dc3484625964cbe2aba49)


In [5]:
model_file

'/zfsauton/home/mbarnes1/data/models/pytorch/fcn32s_from_caffe.pth'

## Load data

In [6]:
val_loader = torch.utils.data.DataLoader(
        torchfcn.datasets.VOC2011ClassSeg(
            root, split='seg11valid', transform=True),
        batch_size=1, shuffle=False,
        num_workers=4, pin_memory=True)
n_class = len(val_loader.dataset.class_names)

## Load model

In [7]:
if osp.basename(model_file).startswith('fcn32s'):
    model = torchfcn.models.FCN32s(n_class=21)
elif osp.basename(model_file).startswith('fcn16s'):
    model = torchfcn.models.FCN16s(n_class=21)
elif osp.basename(model_file).startswith('fcn8s'):
    if osp.basename(model_file).startswith('fcn8s-atonce'):
        model = torchfcn.models.FCN8sAtOnce(n_class=21)
    else:
        model = torchfcn.models.FCN8s(n_class=21)
else:
    raise ValueError
if torch.cuda.is_available():
    model = model.cuda()
print('==> Loading %s model file: %s' %
      (model.__class__.__name__, model_file))
model_data = torch.load(model_file)
try:
    model.load_state_dict(model_data)
except Exception:
    model.load_state_dict(model_data['model_state_dict'])
model.eval()

==> Loading FCN32s model file: /zfsauton/home/mbarnes1/data/models/pytorch/fcn32s_from_caffe.pth


FCN32s(
  (conv1_1): Conv2d (3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(100, 100))
  (relu1_1): ReLU(inplace)
  (conv1_2): Conv2d (64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu1_2): ReLU(inplace)
  (pool1): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), dilation=(1, 1))
  (conv2_1): Conv2d (64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu2_1): ReLU(inplace)
  (conv2_2): Conv2d (128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu2_2): ReLU(inplace)
  (pool2): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), dilation=(1, 1))
  (conv3_1): Conv2d (128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu3_1): ReLU(inplace)
  (conv3_2): Conv2d (256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu3_2): ReLU(inplace)
  (conv3_3): Conv2d (256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu3_3): ReLU(inplace)
  (pool3): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), dilation=(1, 1))
  (conv4_1):

## Evaluate

In [8]:
from torchfcn.trainer import cross_entropy2d, MSEAdjacencyLoss
import torch.nn.functional as F
from torch import nn
mse_loss = MSEAdjacencyLoss(1000)

In [9]:
## Make predictions
visualizations = []
label_trues, label_preds = [], []
loss_cross_entropy = 0
loss_mse = 0
loss_mse_naive_zero = 0  # the loss 
loss_mse_naive_one = 0
n_samples = 0
for batch_idx, (data, target) in tqdm.tqdm(enumerate(val_loader),
                                           total=len(val_loader),
                                           ncols=80, leave=False):
    if torch.cuda.is_available():
        data, target = data.cuda(), target.cuda()
    data, target = Variable(data, volatile=True), Variable(target)
    score = model(data)
    n_samples += score.size(0)
    loss_cross_entropy += cross_entropy2d(score, target, size_average=False)
    loss_mse += mse_loss(F.softmax(score, dim=1), target)
    loss_mse_naive_zero += mse_loss(Variable(score.data.new(score.size()).fill_(0.0)), target)
    all_one_cluster = Variable(score.data.new(score.size()).fill_(0.0))
    all_one_cluster[:, 1, :, :] = 1.0
    loss_mse_naive_one += mse_loss(all_one_cluster, target)
    
    
    imgs = data.data.cpu()
    lbl_pred = score.data.max(1)[1].cpu().numpy()[:, :, :]
    lbl_true = target.data.cpu()
    for img, lt, lp in zip(imgs, lbl_true, lbl_pred):
        img, lt = val_loader.dataset.untransform(img, lt)
        label_trues.append(lt)
        label_preds.append(lp)
        # TODO: Visualizations (get cv2 installed on server)
        #if len(visualizations) < 9:
        #    viz = fcn.utils.visualize_segmentation(
        #        lbl_pred=lp, lbl_true=lt, img=img, n_class=n_class,
        #        label_names=val_loader.dataset.class_names)
        #    visualizations.append(viz)
    if batch_idx > 10:
        break
loss_cross_entropy /= n_samples
loss_mse /= n_samples
loss_mse_naive_zero /= n_samples
loss_mse_naive_one /= n_samples
loss_cross_entropy = loss_cross_entropy.data[0]
loss_mse = loss_mse.data[0]
loss_mse_naive_zero = loss_mse_naive_zero.data[0]
loss_mse_naive_one = loss_mse_naive_one.data[0]

In [10]:
## Save predictions and labels for later
import cPickle as pickle
pickle.dump(label_trues, open( "labels_trues.p", "wb" ) )
pickle.dump(label_preds, open( "labels_preds.p", "wb" ) )

In [11]:
def labels_to_adjacency(labels):
    """
    :param labels: N x M LongTensor Variable, where N is the batch size and M is the number of nodes.
    :return adjacency: N x M x M FloatTensor Variable.
    """
    m = labels.size(1)
    labels = labels.unsqueeze(dim=1).expand(-1, m, -1)  # N x M x M matrix
    adjacency = (labels == labels.transpose(1, 2)).float()
    return adjacency

In [12]:
import random
input = Variable(score.data.new(score.size()).fill_(0.0))
n_nodes = 10000
n, c, h, w = input.size()
total_nodes_per_image = c * h
assert total_nodes_per_image >= n_nodes
random_indices = Variable(target.data.new(random.sample(xrange(total_nodes_per_image), n_nodes * n)).view(n, n_nodes))  # n x n_nodes

# Compute loss
input_subsample = torch.gather(input.view(n, c, -1), 2, random_indices.unsqueeze(dim=1).expand(-1, c, -1))  # N x C x n_nodes
target_subsample = torch.gather(target.view(n, -1), 1, random_indices)  # N x n_nodes
input_adjacency = torch.bmm(input_subsample.transpose(1, 2), input_subsample)  # N x n_nodes x n_nodes
target_adjacency = labels_to_adjacency(target_subsample.view(n, -1))
#off_diagonal_mask = ~torch.eye(self._n_nodes).byte().unsqueeze(dim=0).expand(n, -1, -1)
#loss = self._mse(input_adjacency[off_diagonal_mask], target_adjacency[off_diagonal_mask])  # MSE per edge, excluding self edges
#loss = torch.norm(input_adjacency - target_adjacency, p=2) / (self._n_nodes ** 2)  # Frobenius norm per edge
mse = torch.nn.MSELoss()
loss = mse(input_adjacency, target_adjacency)  # MSE per edge


RuntimeError: cuda runtime error (2) : out of memory at /pytorch/torch/lib/THC/generic/THCStorage.cu:58

In [ ]:
loss

In [ ]:
## Metrics
print('''\
Validation cross-entropy loss per sample: {}
Validation MSE (our predictions) loss per sample: {}
Validation MSE (naively predict all in different clusters, no self edges) loss per sample: {}
Validation MSE (naively predict all in same cluster) loss per sample: {}
'''.format(loss_cross_entropy, loss_mse, loss_mse_naive_zero, loss_mse_naive_one))
metrics = torchfcn.utils.label_accuracy_score(label_trues, label_preds, n_class=n_class)
metrics = np.array(metrics)
metrics *= 100
print('''\
Accuracy: {0}
Accuracy Class: {1}
Mean IU: {2}
FWAV Accuracy: {3}'''.format(*metrics))

In [ ]:
## Visualization
viz = fcn.utils.get_tile_image(visualizations)
skimage.io.imsave('viz_evaluate.png', viz)